In [ ]:
import pandas as pd
import numpy as np

# 3 parts:
# kostenschätzung
# ausgabenberechnung (bilanz berechnen => schätzung vs. tatsächliche kosten)
# kostenvergleich

In [ ]:
from building_cost_analysis.settings import (
    dir_angebote,
    path_gewerke
)
from building_cost_analysis.parser import (
    get_df_gewerke,
    get_offer_file_paths,
    get_offer_file_paths_and_sheets,
    get_df_offer,
    get_df_offers_combined,
    get_list_df_offers
)

from building_cost_analysis.calc import (
    append_rolled_up_sum_to_df,
    get_df_duplicated_positions,
    compare_offers,
    get_missing_positions
)

# get_df_offers_combined
list_paths_sheets = get_offer_file_paths_and_sheets(dir_angebote)
df_gewerke = get_df_gewerke(path_gewerke)

In [ ]:
list_combine_path_sheets = []
for entry in list_paths_sheets:
    if entry['sheet_name'].endswith('Minimum'):
        list_combine_path_sheets.append(entry)
list_combine_path_sheets

In [ ]:
df_offers = get_df_offers_combined(list_combine_path_sheets)
df_combined = append_rolled_up_sum_to_df(df_offers, df_gewerke)

df_combined.loc[df_combined['level'] == 0][['summed_up']].sum() * 1.1# 5 % sicherheitsreserve

In [ ]:
df_combined.loc[df_combined['level'] == 1]

In [ ]:
list_test = list( list_paths_sheets[i] for i in [-3, -1] )
list_test
list_df_offers = get_list_df_offers(list_test)

In [ ]:

list_df_offers = get_list_df_offers(list_paths_sheets[-12:-10])

In [ ]:
def compare_offers(list_df_offers: list, df_gewerke: pd.DataFrame) -> pd.DataFrame:
    # use to compare different offers
    list_summed_up = []
    df_combined = None
    for df_offer in list_df_offers:
        df = append_rolled_up_sum_to_df(df_offer, df_gewerke)
        df = df.loc[df['prices_existing'] == True]
        sheet_name = df['sheet_name'].dropna().iloc[0]
        cols_to_rename = ['summed_up', 'E-Preis', 'Einheit', 'Menge', 'Gesamtpreis']
        rename_dict = {}
        for col_to_rename in cols_to_rename:
            rename_dict[col_to_rename] = col_to_rename + '_' + sheet_name
        df = df.rename(columns=rename_dict)
        list_summed_up.append(df)
        if df_combined is None:
            df_combined = df
        else:
            df_combined = df_combined.combine_first(df)
    return df_combined

df_compare = compare_offers(list_df_offers, df_gewerke)
cols_display = ['summed_up', 'E-Preis', 'Einheit', 'Menge', 'Gesamtpreis']
sheet_names = ['Brunnschmid_1', 'Kracher_1']
cols_final = ['Beschreibung']
for col_display in cols_display:
    for sheet_name in sheet_names:
        cols_final.append(col_display + '_' + sheet_name)
                              
df_compare.loc[df_compare['level'] == 3][cols_final]

In [ ]:
import dash_bootstrap_components as dbc
df = pd.DataFrame(
    {
        "First Name": ["Arthur", "Ford", "Zaphod", "Trillian"],
        "Last Name": ["Dent", "Prefect", "Beeblebrox", "Astra"],
    }
)

table = dbc.Table.from_dataframe(df_offer, striped=True, bordered=True, hover=True)


In [ ]:
def append_rolled_up_sum_to_df_test(df_offers: pd.DataFrame, df_gewerke: pd.DataFrame) -> pd.DataFrame:
    df_offers_int = df_offers.copy()
    df_offers_int['desc_offer'] = df_offers['Beschreibung']
    df_offers_int = df_offers_int.drop(columns=['Beschreibung'])
    df_combined = df_offers_int.combine_first(df_gewerke)
    max_level = df_combined['level'].max()
    list_level_sums = []
    list_ids_sort = []
    for cur_level in range(max_level+1):
        list_ids_sort.append(f'id_{cur_level}')
        list_ids = []
        for lev in range(cur_level + 1):
            list_ids.append(f'id_{lev}')
        df_level = df_combined.loc[df_combined['level'] >= cur_level]
        df_level_count = df_level.groupby(list_ids).count()
        df_level_sum = df_level.groupby(list_ids).sum()
        df_level_sum['summed_up'] = df_level_sum['price_brutto']
        df_level_count['prices_existing'] = False
        df_level_count.loc[df_level_count['price_brutto'] != 0, 'prices_existing'] = True
        df_level_sum = df_level_sum[['summed_up']].combine_first(df_level_count[['prices_existing']])
        df_level_sum['level'] = cur_level
        df_level_sum = df_level_sum.reset_index()
        df_level_sum['ID'] = df_level_sum['id_0'].astype(int).astype(str)
        for lev in range(1, cur_level + 1):
            df_level_sum['ID'] += "." + df_level_sum[f'id_{lev}'].astype(int).astype(str)
        list_level_sums.append(df_level_sum)
    df_rolled_up_sum = pd.DataFrame()

    for df_sum in list_level_sums:
        df_rolled_up_sum = df_rolled_up_sum.combine_first(df_sum.set_index('ID'))
    df_ret = df_combined.combine_first(df_rolled_up_sum[['prices_existing', 'summed_up']])
    df_ret = df_ret.sort_values(list_ids_sort)
    return df_ret


append_rolled_up_sum_to_df_test(list_df_offers[0], df_gewerke).to_csv('./data/test.csv')



In [ ]:
get_df_gewerke